# Database migration rules

In [1]:
import fludashboard as flud
import pandas as pd
import os
import glob

In [2]:
# pandas configuration
pd.set_option('display.max_columns', 99)

In [3]:
def get_filename_from_path(file_path: str):
    """
    """
    return file_path.split(os.path.sep)[-1].split('.')[0]

In [4]:
path_data_files = os.path.join(flud.__path__[0], 'data', '*.csv')

In [5]:
dfs = {}
pks = {}

In [6]:
print('Data files:')
for file_path in glob.glob(path_data_files):
    filename = get_filename_from_path(file_path)
    
    print(filename)
    
    dfs[filename] = pd.read_csv(file_path)

Data files:
mem-report
current_estimated_values
clean_data_epiweek-weekly-incidence_w_situation


/home/xmn/miniconda3/envs/fludashboard/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


historical_estimated_values
mem-typical


## 1. Setting IDs

### 1.1 Datasets

In [7]:
dataset_id = {
    'srag': 1,
    'sragflu': 2,
    'obitoflu': 3,
}

### 1.2 Scale

In [8]:
scale_id = {
    'incidência': 1,
    'casos': 2
}

### 1.3 Situation

In [9]:
situation_id = {
    'unknown': 1, 
    'estimated': 2, 
    'stable': 3
}

### 1.4 Territory Type

In [10]:
territory_type_id = {
    'Estado': 1, 
    'Regional': 2,
    'País': 3
}

### Territory Table

In [11]:
df_territory = pd.DataFrame([
    {'id': '11', 'name': 'Rondônia', 'territory_type': 1},
    {'id': '12', 'name': 'Acre', 'territory_type': 1},
    {'id': '13', 'name': 'Amazonas', 'territory_type': 1},
    {'id': '14', 'name': 'Roraima', 'territory_type': 1},
    {'id': '15', 'name': 'Pará', 'territory_type': 1},
    {'id': '16', 'name': 'Amapá', 'territory_type': 1},
    {'id': '17', 'name': 'Tocantins', 'territory_type': 1},
    {'id': '21', 'name': 'Maranhão', 'territory_type': 1},
    {'id': '22', 'name': 'Piauí', 'territory_type': 1},
    {'id': '23', 'name': 'Ceará', 'territory_type': 1},
    {'id': '24', 'name': 'Rio Grande do Norte', 'territory_type': 1},
    {'id': '25', 'name': 'Paraíba', 'territory_type': 1},
    {'id': '26', 'name': 'Pernambuco', 'territory_type': 1},
    {'id': '27', 'name': 'Alagoas', 'territory_type': 1},
    {'id': '28', 'name': 'Sergipe', 'territory_type': 1},
    {'id': '29', 'name': 'Bahia', 'territory_type': 1},
    {'id': '31', 'name': 'Minas Gerais', 'territory_type': 1},
    {'id': '32', 'name': 'Espírito Santo', 'territory_type': 1},
    {'id': '33', 'name': 'Rio de Janeiro', 'territory_type': 1},
    {'id': '35', 'name': 'São Paulo', 'territory_type': 1},
    {'id': '41', 'name': 'Paraná', 'territory_type': 1},
    {'id': '42', 'name': 'Santa Catarina', 'territory_type': 1},
    {'id': '43', 'name': 'Rio Grande do Sul', 'territory_type': 1},
    {'id': '50', 'name': 'Mato Grosso do Sul', 'territory_type': 1},
    {'id': '51', 'name': 'Mato Grosso', 'territory_type': 1},
    {'id': '52', 'name': 'Goiás', 'territory_type': 1},
    {'id': '53', 'name': 'Distrito Federal', 'territory_type': 1},
    {'id': 'BR', 'name': 'Brasil', 'territory_type': 3},
    {'id': 'RegC', 'name': 'Regional Centro', 'territory_type': 2},
    {'id': 'RegL', 'name': 'Regional Leste', 'territory_type': 2},
    {'id': 'RegN', 'name': 'Regional Norte', 'territory_type': 2},
    {'id': 'RegS', 'name': 'Regional Sul', 'territory_type': 2},
    {'id': 'RegOfN', 'name': 'Norte', 'territory_type': 2},
    {'id': 'RegOfNE', 'name': 'Nordeste', 'territory_type': 2},
    {'id': 'RegOfSE', 'name': 'Sudeste', 'territory_type': 2},
    {'id': 'RegOfCO', 'name': 'Centro-oeste', 'territory_type': 2},
    {'id': 'RegOfS', 'name': 'Sul', 'territory_type': 2},
])

df_territory.set_index('id', inplace=True)

df_territory

,name,territory_type
id,,
11,Rondônia,1
12,Acre,1
13,Amazonas,1
14,Roraima,1
15,Pará,1
16,Amapá,1
17,Tocantins,1
21,Maranhão,1
22,Piauí,1


## 2. current_estimated_values

In [12]:
dataset = 'current_estimated_values'

In [13]:
dfs[dataset].head()

,UF,epiyear,epiweek,SRAG,Tipo,Situation,mean,50%,2.5%,97.5%,L0,L1,L2,L3,Run date,dado,escala
0,11,2009,1,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
1,11,2009,2,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
2,11,2009,3,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
3,11,2009,4,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
4,11,2009,5,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência


In [14]:
migration_rules = {
    'UF': 'state_country',
    'SRAG': 'value',
    'Tipo': 'territory_type',  # Not needed in the table
    'Situation': 'situation',
    '50%': 'median',
    '2.5%': 'ci_lower',
    '97.5%': 'ci_upper',
    'L0': 'low',
    'L1': 'epidemic',
    'L2': 'high',
    'L3': 'very_high',
    'Run date': 'run_date',
    'dado': 'dataset',
    'escala': 'scale'
}

# rename columns
dfs[dataset].rename(
    columns=migration_rules, inplace=True
)

# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].situation = dfs[dataset].situation.map(situation_id)

# remove unnecessary fields
dfs[dataset].drop(['territory_type'], axis=1, inplace=True)

dfs[dataset].head()

,state_country,epiyear,epiweek,value,situation,mean,median,ci_lower,ci_upper,low,epidemic,high,very_high,run_date,dataset,scale
0,11,2009,1,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
1,11,2009,2,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
2,11,2009,3,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
3,11,2009,4,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
4,11,2009,5,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1


In [15]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'epiyear', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

value  situation  mean  median  \
dataset scale state_country epiyear epiweek                                   
1       1     11            2009    1          0.0          3   0.0     0.0   
                                    2          0.0          3   0.0     0.0   
                                    3          0.0          3   0.0     0.0   
                                    4          0.0          3   0.0     0.0   
                                    5          0.0          3   0.0     0.0   

                                             ci_lower  ci_upper  low  \
dataset scale state_country epiyear epiweek                            
1       1     11            2009    1             0.0       0.0  1.0   
                                    2             0.0       0.0  1.0   
                                    3             0.0       0.0  1.0   
                                    4             0.0       0.0  1.0   
                                    5             0.0       0.0  1.0   

                                             epidemic  high  very_high  \
dataset scale state_country epiyear epiweek                              
1       1     11            2009    1             0.0   0.0        0.0   
                                    2             0.0   0.0        0.0   
                                    3             0.0   0.0        0.0   
                                    4             0.0   0.0        0.0   
                                    5             0.0   0.0        0.0   

                                               run_date  
dataset scale state_country epiyear epiweek              
1       1     11            2009    1        2017-10-11  
                                    2        2017-10-11  
                                    3        2017-10-11  
                                    4        2017-10-11  
                                    5        2017-10-11

## 3. historical_estimated_values

In [16]:
dataset = 'historical_estimated_values'

In [17]:
dfs[dataset].head()

,UF,epiyear,epiweek,SRAG,Tipo,Situation,mean,50%,2.5%,97.5%,L0,L1,L2,L3,Run date,base_epiyearweek,base_epiyear,base_epiweek,dado,escala
0,11,2017,7,0.055377,Estado,estimated,0.055377,0.055377,0.055377,0.055377,0.976,0.024,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
1,11,2017,8,0.055377,Estado,estimated,0.055377,0.055377,0.055377,0.110755,0.972,0.028,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
2,11,2017,9,0.000000,Estado,estimated,0.000000,0.000000,0.000000,0.000000,1.000,0.000,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
3,11,2017,10,0.000000,Estado,estimated,0.000000,0.000000,0.000000,0.055377,0.996,0.004,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
4,11,2017,11,0.055377,Estado,estimated,0.055377,0.055377,0.055377,0.110755,0.964,0.036,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência


In [18]:
migration_rules = {
    'UF': 'state_country',
    'SRAG': 'value',
    'Tipo': 'territory_type',  # Not needed in the table
    'Situation': 'situation',
    '50%': 'median',
    '2.5%': 'ci_lower',
    '97.5%': 'ci_upper',
    'L0': 'low',
    'L1': 'epidemic',
    'L2': 'high',
    'L3': 'very_high',
    'Run date': 'run_date',
    'dado': 'dataset',  # or origin
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].situation = dfs[dataset].situation.map(situation_id)

# remove unnecessary fields
dfs[dataset].drop(['territory_type'], axis=1, inplace=True)

dfs[dataset].head()

,state_country,epiyear,epiweek,value,situation,mean,median,ci_lower,ci_upper,low,epidemic,high,very_high,run_date,base_epiyearweek,base_epiyear,base_epiweek,dataset,scale
0,11,2017,7,0.055377,2,0.055377,0.055377,0.055377,0.055377,0.976,0.024,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
1,11,2017,8,0.055377,2,0.055377,0.055377,0.055377,0.110755,0.972,0.028,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
2,11,2017,9,0.000000,2,0.000000,0.000000,0.000000,0.000000,1.000,0.000,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
3,11,2017,10,0.000000,2,0.000000,0.000000,0.000000,0.055377,0.996,0.004,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
4,11,2017,11,0.055377,2,0.055377,0.055377,0.055377,0.110755,0.964,0.036,0.0,0.0,2017-10-11,2017W23,2017,23,1,1


In [19]:
# primary_keys
pks[dataset] = [
    'dataset', 'scale', 'state_country', 
    'base_epiyear', 'base_epiweek', 
    'epiyear', 'epiweek'
]

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

value  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       situation  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek              
1       1     11            2017         23           2017    7                2   
                                                              8                2   
                                                              9                2   
                                                              10               2   
                                                              11               2   

                                                                           mean  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                         median  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       ci_lower  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       ci_upper  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.110755   
                                                              9        0.000000   
                                                              10       0.055377   
                                                              11       0.110755   

                                                                         low  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek          
1       1     11            2017         23           2017    7        0.976   
                                                              8        0.972   
                                                              9        1.000   
                                                              10       0.996   
                                                              11       0.964   

              

## 4. clean_data_epiweek-weekly-incidence_w_situation

In [20]:
dataset = 'clean_data_epiweek-weekly-incidence_w_situation'

In [21]:
dfs[dataset].head()

,0-4 anos,10-19 anos,2-4 anos,20-29 anos,30-39 anos,40-49 anos,5-9 anos,50-59 anos,60+ anos,< 2 anos,DELAYED,FLU_A,FLU_B,INCONCLUSIVE,Idade desconhecida,NEGATIVE,NOTTESTED,OTHERS,POSITIVE_CASES,SRAG,Situation,TESTING_IGNORED,Tipo,UF,Unidade da Federação,VSR,dado,epiweek,epiyear,epiyearweek,escala,sexo
0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,1,2009,2009W01,incidência,F
1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,1,2009,2009W01,incidência,M
2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,1,2009,2009W01,incidência,Total
3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,2,2009,2009W02,incidência,F
4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,2,2009,2009W02,incidência,M


In [22]:
dfs[dataset].keys()

Index(['0-4 anos', '10-19 anos', '2-4 anos', '20-29 anos', '30-39 anos',
       '40-49 anos', '5-9 anos', '50-59 anos', '60+ anos', '< 2 anos',
       'DELAYED', 'FLU_A', 'FLU_B', 'INCONCLUSIVE', 'Idade desconhecida',
       'NEGATIVE', 'NOTTESTED', 'OTHERS', 'POSITIVE_CASES', 'SRAG',
       'Situation', 'TESTING_IGNORED', 'Tipo', 'UF', 'Unidade da Federação',
       'VSR', 'dado', 'epiweek', 'epiyear', 'epiyearweek', 'escala', 'sexo'],
      dtype='object')

In [23]:
migration_rules = {
    '0-4 anos': '0_4_years',
    '10-19 anos': '10_19_years',
    '2-4 anos': '2_4_years',
    '20-29 anos': '20_29_years',
    '30-39 anos': '30_39_years',
    '40-49 anos': '40_49_years',
    '5-9 anos': '5_9_years',
    '50-59 anos': '50_59_years',
    '60+ anos': '60_years_or_more',
    '< 2 anos': 'lt_2_years',
    'DELAYED': 'delayed',
    'FLU_A': 'flu_a',
    'FLU_B': 'flu_b',
    'INCONCLUSIVE': 'inconclusive',
    'Idade desconhecida': 'unknown_age',
    'NEGATIVE': 'negative',
    'NOTTESTED': 'not_tested',
    'OTHERS': 'others',
    'POSITIVE_CASES': 'positive_cases',
    'SRAG': 'value',
    'Situation': 'situation',
    'TESTING_IGNORED': 'testing_ignored',
    'Tipo': 'territory_type',  # Not needed in the table
    'UF': 'state_country',
    'Unidade da Federação': 'state_country_name',  # Not needed in the table
    'VSR': 'vsr',
    'dado': 'dataset',
    'escala': 'scale',
    'sexo': 'gender'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].situation = dfs[dataset].situation.map(situation_id)

# remove unnecessary fields
dfs[dataset].drop([
    'state_country_name', 'territory_type'
], axis=1, inplace=True)


dfs[dataset].head()

,0_4_years,10_19_years,2_4_years,20_29_years,30_39_years,40_49_years,5_9_years,50_59_years,60_years_or_more,lt_2_years,delayed,flu_a,flu_b,inconclusive,unknown_age,negative,not_tested,others,positive_cases,value,situation,testing_ignored,state_country,vsr,dataset,epiweek,epiyear,epiyearweek,scale,gender
0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,1,2009,2009W01,1,F
1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,1,2009,2009W01,1,M
2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,1,2009,2009W01,1,Total
3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,2,2009,2009W02,1,F
4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,2,2009,2009W02,1,M


In [24]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'epiyear', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

0_4_years  10_19_years  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              0.0          0.0   
                                    1              0.0          0.0   
                                    1              0.0          0.0   
                                    2              0.0          0.0   
                                    2              0.0          0.0   

                                             2_4_years  20_29_years  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              NaN          0.0   
                                    1              NaN          0.0   
                                    1              NaN          0.0   
                                    2              NaN          0.0   
                                    2              NaN          0.0   

                                             30_39_years  40_49_years  \
dataset scale state_country epiyear epiweek                             
1       1     11            2009    1                0.0          0.0   
                                    1                0.0          0.0   
                                    1                0.0          0.0   
                                    2                0.0          0.0   
                                    2                0.0          0.0   

                                             5_9_years  50_59_years  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              0.0          0.0   
                                    1              0.0          0.0   
                                    1              0.0          0.0   
                                    2              0.0          0.0   
                                    2              0.0          0.0   

                                             60_years_or_more  lt_2_years  \
dataset scale state_country epiyear epiweek                                 
1       1     11            2009    1                     0.0         NaN   
                                    1                     0.0         NaN   
                                    1                     0.0         NaN   
                                    2                     0.0         NaN   
                                    2                     0.0         NaN   

                                             delayed  flu_a  flu_b  \
dataset scale state_country epiyear epiweek                          
1       1     11            2009    1            0.0    0.0    0.0   
                                    1            0.0    0.0    0.0   
                                    1            0.0    0.0    0.0   
                                    2            0.0    0.0    0.0   
                                    2            0.0    0.0    0.0   

                                             inconclusive  unknown_age  \
dataset scale state_country epiyear epiweek                              
1       1     11            2009    1                 0.0          NaN   
                                    1                 0.0          NaN   
                                    1                 0.0          NaN   
                                    2                 0.0          NaN   
                                    2                 0.0          NaN   

                                             negative  not_tested  others  \
dataset scale state_country epiyear epiweek                                 
1       1     11            2009    1             0.0         0.0     0.0   
                                    1             0.0         0.0     0.0   
                                    1             0.0         0.0     0.0   
                                    2             0.0         0.0     0.0   
                           

## 5. mem-report

In [25]:
dataset = 'mem-report'

In [26]:
dfs[dataset].head()

,UF,População,Média geométrica do pico de infecção das temporadas regulares,região de baixa atividade típica,limiar pré-epidêmico,intensidade alta,intensidade muito alta,SE típica do início do surto,"SE típica do início do surto - IC inferior (2,5%)","SE típica do início do surto - IC superior (97,5%)",duração típica do surto,"duração típica do surto - IC inferior (2,5%)","duração típica do surto - IC superior (97,5%)",temporadas utilizadas para os corredores endêmicos,ano,Unidade da Federação,Tipo,dado,escala
0,11,1805788,0.803458,0,0.103162,0.863224,1.168160,12.0,9.0,22.0,8.0,4.0,15.0,"2010, 2013, 2014, 2016",2017,Rondônia,Estado,srag,incidência
1,12,829619,0.953503,0,0.685939,1.783885,3.053363,24.0,5.0,49.0,6.0,3.0,16.0,"2010, 2011, 2012, 2013, 2014, 2015, 2016",2017,Acre,Estado,srag,incidência
2,13,4063614,0.119378,0,0.042171,0.165851,0.212193,12.0,4.0,25.0,12.0,6.0,13.0,"2010, 2012, 2013, 2014, 2015, 2016",2017,Amazonas,Estado,srag,incidência
3,14,522636,0.473615,0,0.414595,0.593964,0.801757,26.0,13.0,50.0,6.0,3.0,10.0,"2012, 2013, 2014, 2015, 2016",2017,Roraima,Estado,srag,incidência
4,15,8366628,0.256230,0,0.083865,0.466878,0.859727,12.0,10.0,18.0,10.0,5.0,13.0,"2012, 2014, 2015, 2016",2017,Pará,Estado,srag,incidência


In [27]:
dfs[dataset].keys()

Index(['UF', 'População',
       'Média geométrica do pico de infecção das temporadas regulares',
       'região de baixa atividade típica', 'limiar pré-epidêmico',
       'intensidade alta', 'intensidade muito alta',
       'SE típica do início do surto',
       'SE típica do início do surto - IC inferior (2,5%)',
       'SE típica do início do surto - IC superior (97,5%)',
       'duração típica do surto',
       'duração típica do surto - IC inferior (2,5%)',
       'duração típica do surto - IC superior (97,5%)',
       'temporadas utilizadas para os corredores endêmicos', 'ano',
       'Unidade da Federação', 'Tipo', 'dado', 'escala'],
      dtype='object')

In [28]:
migration_rules = {
    'UF': 'state_country',
    'População': 'population',
    'Média geométrica do pico de infecção das temporadas regulares': 'geom_average_peak',
    'região de baixa atividade típica': 'low_activiy_region',
    'limiar pré-epidêmico': 'pre_epidemic_threshold',
    'intensidade alta': 'high_threshold',
    'intensidade muito alta': 'very_high_threshold',
    'SE típica do início do surto': 'epi_start',
    'SE típica do início do surto - IC inferior (2,5%)': 'epi_start_ci_lower',
    'SE típica do início do surto - IC superior (97,5%)': 'epi_start_ci_upper',
    'duração típica do surto': 'epi_duration',
    'duração típica do surto - IC inferior (2,5%)': 'epi_duration_ci_lower',
    'duração típica do surto - IC superior (97,5%)': 'epi_duration_ci_upper',
    'temporadas utilizadas para os corredores endêmicos': 'regular_seasons',
    'ano': 'year',
    'Unidade da Federação': 'state_country_name',  # Not needed in the table
    'Tipo': 'territory_type',  # Not needed in the table
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)

# remove unnecessary fields
dfs[dataset].drop([
    'state_country_name', 'territory_type'
], axis=1, inplace=True)

dfs[dataset].head()

,state_country,population,geom_average_peak,low_activiy_region,pre_epidemic_threshold,high_threshold,very_high_threshold,epi_start,epi_start_ci_lower,epi_start_ci_upper,epi_duration,epi_duration_ci_lower,epi_duration_ci_upper,regular_seasons,year,dataset,scale
0,11,1805788,0.803458,0,0.103162,0.863224,1.168160,12.0,9.0,22.0,8.0,4.0,15.0,"2010, 2013, 2014, 2016",2017,1,1
1,12,829619,0.953503,0,0.685939,1.783885,3.053363,24.0,5.0,49.0,6.0,3.0,16.0,"2010, 2011, 2012, 2013, 2014, 2015, 2016",2017,1,1
2,13,4063614,0.119378,0,0.042171,0.165851,0.212193,12.0,4.0,25.0,12.0,6.0,13.0,"2010, 2012, 2013, 2014, 2015, 2016",2017,1,1
3,14,522636,0.473615,0,0.414595,0.593964,0.801757,26.0,13.0,50.0,6.0,3.0,10.0,"2012, 2013, 2014, 2015, 2016",2017,1,1
4,15,8366628,0.256230,0,0.083865,0.466878,0.859727,12.0,10.0,18.0,10.0,5.0,13.0,"2012, 2014, 2015, 2016",2017,1,1


In [29]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'year']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

population  geom_average_peak  \
dataset scale state_country year                                  
1       1     11            2017     1805788           0.803458   
              12            2017      829619           0.953503   
              13            2017     4063614           0.119378   
              14            2017      522636           0.473615   
              15            2017     8366628           0.256230   

                                  low_activiy_region  pre_epidemic_threshold  \
dataset scale state_country year                                               
1       1     11            2017                   0                0.103162   
              12            2017                   0                0.685939   
              13            2017                   0                0.042171   
              14            2017                   0                0.414595   
              15            2017                   0                0.083865   

                                  high_threshold  very_high_threshold  \
dataset scale state_country year                                        
1       1     11            2017        0.863224             1.168160   
              12            2017        1.783885             3.053363   
              13            2017        0.165851             0.212193   
              14            2017        0.593964             0.801757   
              15            2017        0.466878             0.859727   

                                  epi_start  epi_start_ci_lower  \
dataset scale state_country year                                  
1       1     11            2017       12.0                 9.0   
              12            2017       24.0                 5.0   
              13            2017       12.0                 4.0   
              14            2017       26.0                13.0   
              15            2017       12.0                10.0   

                                  epi_start_ci_upper  epi_duration  \
dataset scale state_country year                                     
1       1     11            2017                22.0           8.0   
              12            2017                49.0           6.0   
              13            2017                25.0          12.0   
              14            2017                50.0           6.0   
              15            2017                18.0          10.0   

                                  epi_duration_ci_lower  \
dataset scale state_country year                          
1       1     11            2017                    4.0   
              12            2017                    3.0   
              13            2017                    6.0   
              14            2017                    3.0   
              15            2017                    5.0   

                                  epi_duration_ci_upper  \
dataset scale state_country year                          
1       1     11            2017                   15.0   
              12            2017                   16.0   
              13            2017                   13.0   
              14            2017                   10.0   
              15            2017                   13.0   

                                                           regular_seasons  
dataset scale state_country year                                            
1       1     11            2017                    2010, 2013, 2014, 2016  
              12            2017  2010, 2011, 2012, 2013, 2014, 2015, 2016  
              13            2017        2010, 2012, 2013, 2014, 2015, 2016  
              14            2017              2012, 2013, 2014, 2015, 2016  
              15            2017                    2012, 2014, 2015, 2016

## 6. mem-typical

In [30]:
dataset = 'mem-typical'

In [31]:
dfs[dataset].head()

,UF,População,epiweek,corredor baixo,corredor mediano,corredor alto,ano,Unidade da Federação,Tipo,dado,escala
0,11,1805788,1,0.0,0.013703,0.041109,2017,Rondônia,Estado,srag,incidência
1,11,1805788,2,0.0,0.000000,0.000000,2017,Rondônia,Estado,srag,incidência
2,11,1805788,3,0.0,0.013703,0.041109,2017,Rondônia,Estado,srag,incidência
3,11,1805788,4,0.0,0.028790,0.087644,2017,Rondônia,Estado,srag,incidência
4,11,1805788,5,0.0,0.000000,0.000000,2017,Rondônia,Estado,srag,incidência


In [32]:
dfs[dataset].keys()

Index(['UF', 'População', 'epiweek', 'corredor baixo', 'corredor mediano',
       'corredor alto', 'ano', 'Unidade da Federação', 'Tipo', 'dado',
       'escala'],
      dtype='object')

In [33]:
migration_rules = {
    'UF': 'state_country',
    'População': 'population',
    'corredor baixo': 'low',
    'corredor mediano': 'median',
    'corredor alto': 'high',
    'ano': 'year',
    'Unidade da Federação': 'state_country_name',  # Not needed in the table
    'Tipo': 'territory_type',  # Not needed in the table
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)

# remove unnecessary fields
dfs[dataset].drop([
    'state_country_name', 'territory_type'
], axis=1, inplace=True)

dfs[dataset].head()

,state_country,population,epiweek,low,median,high,year,dataset,scale
0,11,1805788,1,0.0,0.013703,0.041109,2017,1,1
1,11,1805788,2,0.0,0.000000,0.000000,2017,1,1
2,11,1805788,3,0.0,0.013703,0.041109,2017,1,1
3,11,1805788,4,0.0,0.028790,0.087644,2017,1,1
4,11,1805788,5,0.0,0.000000,0.000000,2017,1,1


In [34]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'year', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

population  low    median      high
dataset scale state_country year epiweek                                     
1       1     11            2017 1           1805788  0.0  0.013703  0.041109
                                 2           1805788  0.0  0.000000  0.000000
                                 3           1805788  0.0  0.013703  0.041109
                                 4           1805788  0.0  0.028790  0.087644
                                 5           1805788  0.0  0.000000  0.000000

@TODO convert regions id to numerical value

## SQL Migration

In [35]:
# @TODO treat run_date
def to_sql(df: pd.DataFrame, name: str):
    sql = 'CREATE TABLE %(table_name)s (\n%(fields)s\n);\n' 
    fields = ''
    header = ''
    
    dict_type = {
        'float64': 'DOUBLE',
        'int64': 'INTEGER', 
        'object': 'VARCHAR'
    }
    pks = list(df.index.names)
    
    df = df.reset_index()
    name_max_len = len(max(df.keys(), key=lambda v: len(v)))
    
    for k in df.keys():
        field_settings = {
            'name': k.ljust(name_max_len, ' '),
            'type': dict_type[str(df[k].dtype)].ljust(7, ' '),
            'null': 'NOT NULL' if (df[k].isnull() == False).all() else 'NULL'
        }
        
        if field_settings['type'] == 'varchar':
            field_settings['type'] += '(%s)' % int(df[k].str.len().max())
            
        fields += header + ('    %(name)s    %(type)s    %(null)s' % field_settings)
        header = ',\n'
        
    fields += header + ('    PRIMARY KEY (%s)' % ', '.join(pks))
    print(sql % {'table_name': name, 'fields': fields})

In [36]:
for k in dfs.keys():
    k_new = k.replace('-', '_')
    to_sql(dfs[k], k_new)

CREATE TABLE mem_report (
    dataset                   INTEGER    NOT NULL,
    scale                     INTEGER    NOT NULL,
    state_country             VARCHAR    NOT NULL,
    year                      INTEGER    NOT NULL,
    population                INTEGER    NOT NULL,
    geom_average_peak         DOUBLE     NULL,
    low_activiy_region        INTEGER    NOT NULL,
    pre_epidemic_threshold    DOUBLE     NOT NULL,
    high_threshold            DOUBLE     NOT NULL,
    very_high_threshold       DOUBLE     NOT NULL,
    epi_start                 DOUBLE     NULL,
    epi_start_ci_lower        DOUBLE     NULL,
    epi_start_ci_upper        DOUBLE     NULL,
    epi_duration              DOUBLE     NULL,
    epi_duration_ci_lower     DOUBLE     NULL,
    epi_duration_ci_upper     DOUBLE     NULL,
    regular_seasons           VARCHAR    NULL,
    PRIMARY KEY (dataset, scale, state_country, year)
);

CREATE TABLE current_estimated_values (
    dataset          INTEGER    NOT NULL,